# Giskard Robot playground

## 1. Parameters and Helper function

In [2]:
import os
import subprocess
import rospy
from ipywidgets import Button, Layout, GridBox, Box
from IPython.display import display, Markdown, IFrame
from sidecar import Sidecar

LAUNCH_FILE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../launch"))
SIDECAR = None
LAUNCH_PROCESS = None

try:
    RIVZWEB = IFrame(src=rospy.get_param('/rvizweb/jupyter_proxy_url'), width='100%', height='100%')
except Exception as e:
    RIVZWEB = None

try:
    HAS_DISPLAY = 'true' if os.environ['DISPLAY'] != ':100' else 'false'
except KeyError:
    HAS_DISPLAY = 'false'

def open_rvizweb():
    global SIDECAR
    if SIDECAR is not None:
        SIDECAR.close()
    if RIVZWEB is not None:
        SIDECAR = Sidecar(title='RVIZWEB', anchor='right')
        with SIDECAR:
            display(RIVZWEB)

def launch_robot(robot):
    global LAUNCH_PROCESS
    if LAUNCH_PROCESS is not None:
        LAUNCH_PROCESS.terminate()
        LAUNCH_PROCESS.wait()
    launchfile = os.path.join(LAUNCH_FILE_DIR, f"{robot}_standalone.launch")
    command = f"roslaunch {launchfile} gui:={HAS_DISPLAY}"
    LAUNCH_PROCESS = subprocess.Popen(['/bin/bash', '-c', command],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL, shell=False)
    open_rvizweb()

def create_button(name):
    btn = Button(
        description=name.upper(),
        layout=Layout(width='auto', height='50px'),
        style={'font_size':'1rem'},
        tooltip=f"Launch robot: {name}"
    )
    btn.on_click(lambda b: launch_robot(name))
    return btn

## 2.Launch a robot
>Note: The launch process will run in foreground and block
the execution of the the following code cells!

In [3]:
robot_list = ['pr2', 'tiago', 'hsr', 'donbot', 'tracy', 'boxy', 'stretch', 'armar']
button_list = [create_button(robot) for robot in robot_list]
print('Click the buttons to launch a robot.')
display(GridBox(button_list, layout=Layout(grid_template_columns="repeat(4, 25%)")))

Click the buttons to launch a robot.


GridBox(children=(Button(description='PR2', layout=Layout(height='50px', width='auto'), style=ButtonStyle(font…

## 3. Control the robot with the Giskard Python Interface
Source: https://github.com/SemRoCo/giskardpy/wiki/Old-Python-Interface

In [4]:
from giskardpy.python_interface import GiskardWrapper

In [6]:
giskard_wrapper = GiskardWrapper()
giskard_wrapper.set_joint_goal({'torso_lift_joint':0.2})
giskard_wrapper.set_cart_goal('torso_lift_link', 'r_gripper_tool_frame', PoseStamped())
giskard_wrapper.plan_and_execute(wait=True)
giskard_wrapper.add_cmd()

ROSInitException: time is not initialized. Have you called init_node()?

## Todos:
- Launch robot in a background subprocess. 
- Add Giskard Python Interface demo
- Blockly control
- Fix boxy arms are not visible in Rvizweb 
- Rvizweb scene refresh and sidecar display
- Armar description not found

In [6]:
!date

Tue 23 Jan 2024 11:37:38 AM UTC
[INFO] [1706009772.332119]: [/giskard_interactive_marker]: got interactive goal update
[INFO] [1706009782.083323]: [/giskard_interactive_marker]: got interactive goal update
[/unnamed]: Found these qp solvers: ['qpalm']
[INFO] [1706009733.900584]: [/giskard]: Using betterpybullet for collision checking.
[INFO] [1706009734.012288]: [/giskard]: loading self collision matrix: /home/jovyan/workspace/ros/src/giskardpy/self_collision_matrices/iai/boxy_description.srdf
[INFO] [1706009734.076724]: [/giskard]: Loaded self collision matrix: /home/jovyan/workspace/ros/src/giskardpy/self_collision_matrices/iai/boxy_description.srdf
[INFO] [1706009734.107881]: [/giskard]: The following joints are non-fixed according to the urdf, but not flagged as controlled: {'boxy_description/right_gripper_joint', 'boxy_description/left_gripper_base_gripper_left_joint', 'boxy_description/left_gripper_joint', 'boxy_description/right_gripper_base_gripper_left_joint'}.
[INFO] [1706009